<a href="https://colab.research.google.com/github/phenix-project/Colabs/blob/main/CCTBX_Demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction to CCTBX

The CCTBX project (computational crystallography toolbox) initially started as a library containing algorithms for the handling of unit cells, space groups, and atomic scatterers. Since then it has grown into a large project to support a variety of structural biology workflows, including x-ray and neutron crystallography, cryo-electron microscopy, and x-ray free electron lasers. 

The CCTBX library is written in Python, with many of the lower level algorithms accelerated using C++ via the Boost Python bindings.

The principal high-level cctbx objects are:

  - Data Manager: Read and write files and keep track of them
  - Model Manager: A molecular model or models and restraints that go with them
  - Map Manager: A 3D map or maps and information about gridding and map symmetry
  - Map Model Manager: One or more related maps and models as a group
  - F-Model Manager: Combines a model and a related Miller array (map coefficeints, or other data for each index of a map)

\
These high-level cctbx objects are composed of lower-level objects that implement various concepts in macromolecular crystallography, which are in turn composed of CCTBX flex arrays for efficient computation of numerical data. Finally, user defined parameters are managed and shared betwen objects using the CCTBX Phil module. 

\
The examples presented here aim to illustrate working with the core CCTBX data types using example data. Much of the content is adapted from the official CCTBX documentation: http://cci.lbl.gov/docs/cctbx/

\
Many of the examples are drawn directly from those created by Dorothee Liebschner and Tom Terwilliger. Additional credit goes to Billy Poon for packaging CCTBX as a conda package, and to Billy and Claudia for helping to get CCTBX working in a Colab notebook.

#Installation
### Installation is in three steps:
####    1.  the Anaconda Python package manager, CCTBX, and py3Dmol
####    2. Retrieve the chemical data repositories and some example data
####    3. Restart the Runtime and load CCTBX environment variables

In [ ]:
#@title Install software
#@markdown Please execute this cell by pressing the _Play_ button 
#@markdown on the left. Installation may take a few minutes.
#@markdown Double click on this text to show/hide the installation script

from IPython.utils import io
import tqdm.notebook

total = 50

MINICONDA_PREFIX="/usr/local"
MINICONDA_INSTALLER_SCRIPT="Miniconda3-py37_4.9.2-Linux-x86_64.sh"

with tqdm.notebook.tqdm(total=total) as pbar:
    with io.capture_output() as captured:
      # install anaconda
      %shell wget https://repo.continuum.io/miniconda/$MINICONDA_INSTALLER_SCRIPT
      pbar.update(10)
      %shell chmod +x {MINICONDA_INSTALLER_SCRIPT}
      %shell ./{MINICONDA_INSTALLER_SCRIPT} -b -f -p {MINICONDA_PREFIX}
      %shell conda install -c conda-forge cctbx -y
      pbar.update(30)
      
      # provide location of libtbx environment
      %shell ln -s /usr/local/share/cctbx /usr/share/cctbx

      # replace system libstdc++ with conda version
      # restart runtime after running
      # to keep changes persistent, modify the filesystem, not environment variables
      %shell sudo cp /usr/local/lib/libstdc++.so.6.0.29 /usr/lib/x86_64-linux-gnu/libstdc++.so.6 


      # non cctbx installs
      %shell pip install py3dmol
      %shell apt install -y subversion

      # python scripts for py3dmol + cctbx
      %shell mkdir cctbx_py3dmol
      %shell cd cctbx_py3dmol
      %shell wget https://raw.githubusercontent.com/cschlick/cctbx-notebooks/master/cctbx_py3dmol_wrapper.py
      %shell wget https://raw.githubusercontent.com/cschlick/cctbx-notebooks/master/cubetools.py
      %shell cd ../
      pbar.update(10)

      

In [ ]:
#@title Download data
#@markdown Please execute this cell by pressing the _Play_ button 
#@markdown on the left. This should take a minute.
%%capture
%%bash

# get chemical data
cd  /usr/local/lib/python3.7/site-packages
mkdir chem_data
cd chem_data

svn --quiet --non-interactive --trust-server-cert co https://github.com/phenix-project/geostd.git/trunk geostd
svn --quiet --non-interactive --trust-server-cert co https://github.com/rlabduke/mon_lib.git/trunk mon_lib
svn --quiet --non-interactive --trust-server-cert export https://github.com/rlabduke/reference_data.git/trunk/Top8000/Top8000_rotamer_pct_contour_grids rotarama_data
svn --quiet --non-interactive --trust-server-cert --force export https://github.com/rlabduke/reference_data.git/trunk/Top8000/Top8000_ramachandran_pct_contour_grids rotarama_data
svn --quiet --non-interactive --trust-server-cert co https://github.com/rlabduke/reference_data.git/trunk/Top8000/Top8000_cablam_pct_contour_grids cablam_data
svn --quiet --non-interactive --trust-server-cert co https://github.com/rlabduke/reference_data.git/trunk/Top8000/rama_z rama_z

# update rotamer and cablam cache
/usr/local/bin/mmtbx.rebuild_rotarama_cache
/usr/local/bin/mmtbx.rebuild_cablam_cache


# get probe
git clone https://github.com/cschlick/cctbx-notebooks.git
mkdir -p /usr/local/share/cctbx/probe/exe
cp cctbx-notebooks/probe /usr/local/share/cctbx/probe/exe
chmod +x /usr/local/share/cctbx/probe/exe/probe
mkdir -p /usr/lib/python3.7/site-packages/probe
mkdir -p /usr/lib/python3.7/site-packages/reduce

# tutorial data
cd /content/ 
wget https://gitcdn.link/repo/phenix-lbl/cctbx_tutorial_files/master/2019_melk/1aba_pieces.pdb
wget https://gitcdn.link/repo/phenix-lbl/cctbx_tutorial_files/master/2019_melk/1aba_model.pdb
wget https://gitcdn.link/repo/phenix-lbl/cctbx_tutorial_files/master/2019_melk/4zyp.mtz
wget https://gitcdn.link/repo/phenix-lbl/cctbx_tutorial_files/master/2019_melk/1aba_reflections.mtz
wget https://gitcdn.link/repo/phenix-lbl/cctbx_tutorial_files/master/2019_melk/resname_mix.pdb


**WAIT!**...don't click the next button yet...Go to "Runtime" and select "Restart Runtime" first.  Then click the button below as noted.

In [ ]:
#@title Set environment variables and do imports
#@markdown Execute this cell by clicking the run button the left
%%capture
#  Manually update sys.path
%env PYTHONPATH="$/env/python:/usr/local/lib:/usr/local/lib/python3.7/lib-dynload:/usr/local/lib/python3.7/site-packages"

# add conda paths to sys.path
import sys, os
sys.path.extend(['/usr/local/lib', '/usr/local/lib/python3.7/lib-dynload', '/usr/local/lib/python3.7/site-packages',"/content/cctbx_py3dmol"])
os.environ["MMTBX_CCP4_MONOMER_LIB"]="/usr/local/lib/python3.7/site-packages/chem_data/geostd"
os.environ["LIBTBX_BUILD"]= "/usr/local/share/cctbx"

import cctbx
import libtbx
libtbx.env.add_repository("/usr/local/lib/python3.7/site-packages/")
libtbx.env.module_dist_paths['probe']=""

from cubetools import *
from cctbx_py3dmol_wrapper import CCTBX3dmolWrapper



#### Test
We should (hopefully) not see any errors. Use the Run button or Shift+Enter to execute the cells

In [ ]:
import cctbx
from scitbx.array_family import flex
a = flex.random_double(100)
print(flex.min(a))

We can also run linux shell commands from Colab cells, either using the %shell syntax (using during installation), or the ! prefix for a single line. Lets check that the downloads were successful and that we have some files to work with. For example, we hope to see 1aba_model.pdb in the current working directory.

In [ ]:
! pwd

In [ ]:
! ls

# File IO via the DataManager

When you want to perform some actions on data or a model, you have to start with reading the file. Once the action is done, you probably want to write the files. 

To do this conveniently, you can use the cctbx DataManager. The DataManager lets you read and write files describing atomic models, restraints, reflection data files, symmetry files, sequence files, and MRC/CCP4 format map files.

In [ ]:
from iotbx.data_manager import DataManager

dm = DataManager()
filename_pdb = "1aba_model.pdb"
dm.process_model_file(filename_pdb)

The data manager has now read our file. If desired, a single data manager instance can read many files and store them simultaneously. To access a specific file, we can request it using the filename as the key...

In [ ]:
model = dm.get_model(filename=filename_pdb)

The model object is an instance of the CCTBX Model Manager. The Model Manager stores all information related to a molecular model. This includes the molecular composition, space group and symmetry information, information about geometry restraints, and any metadata that was available in the input file.

\
We will use the py3dmol package: https://3dmol.org/ to display structures in the notebook. We can customize the depictions by writing a custom show function (already downloaded during the data download step). Run the next cell to import the show function and show your model. You can manipulate the model with your mouse...

In [ ]:
from cctbx_py3dmol_wrapper import CCTBX3dmolWrapper
show = CCTBX3dmolWrapper()
show(model)

To save a model to the disk, we also use the CCTBX data manager. One of the many advantages of the data manager is the ability to handle multiple file formats transparently.

In [ ]:
filename_cif = "1aba_model.cif"
dm.write_model_file(model,filename=filename_cif,format="cif")

Lets check to make sure it worked

In [ ]:
! head 1aba_model.cif -n 100

# Models

## Model Input

The model manager object provides a a container for all information related to a model. The categories of data inclue the model metadata, the model composition, and geometry geometry restraints for the model. By maintaining a clear separation between these categories of data, we can avoid passing unneccessary information to low level functions.

\
First we will consider the model metadata which is stored as a model input object. In contrast to the model composition which is highly structured data, the model input can vary substantially depending on the source of the file and what the authors chose to include. Lets take a look at the model input for this file. Execute the cell below to get the model_input object, then the button below that to print out some characteristics.|

In [ ]:
model_input= model.get_model_input()

Some fields are structured, and are directly accessible by member functions

In [ ]:
print(model_input.file_type())
print(model_input.get_experiment_type())
print(model_input.get_program_name())
print(model_input.deposition_date())
print(model_input.resolution())
print(model_input.sequence_from_SEQRES())


The entire contents of the file are retained, and can be parsed to extract miscellaneous fields. Note that many fields relating to the data are not populated. However, we can determine them later when we look at the data for this structure.

In [ ]:
# Get the whole title section of the pdb
print("\n".join(model_input.title_section()))

In [ ]:
# print the remark section
print("\n".join(model_input.remark_section())[:2000])

## Crystal symmetry

The crystal symmetry object stores structured data about the unit cell and space group

In [ ]:
cs = model.crystal_symmetry()
print(cs)

## Model composition
The core componnet of the Model Manager object is the information about molecular composition, stored as a hierarchy object. The hierarchy is written as a C++ object for speed, and structured to reflect the hierarchical nature of macromolecules. It also aims to be independent of the original file format. That is, the hierarchy object doesn't know or care if the original file was a PDB file, mmCIF file, or any other molecular file format.

\
Lets take a closer look at the composition of this protein

In [ ]:
hierarchy = model.get_hierarchy()
print(hierarchy.composition())

#### Group args
The composition method returns a group_args object. This is a CCTBX object that is commonly returned as the result from CCTBX programs and methods. It essentially functions the same as a dictionary, holding data as a set of key,value pairs. The advantage of the group_args object is that the keys can be accessed using dot syntax, ie: 



In [ ]:
result = hierarchy.composition()
n_atoms = result.n_atoms
print(n_atoms)

This is a convienent way to work with program results, especially if the results contain multiple nested group_args. Alternatively, the fields can be accessed by iterating through the keys

In [ ]:
for key in result.keys():
  print(key,":",result.get(key))

### Architecture of the hierarchy object

```
model(s)
  id
  chain(s)
    id
    residue_group(s)
      resseq
      icode
      atom_group(s)
        altloc,
        resname
          atom(s)
            name
            element
            charge
            xyz
            occ
            b
```
 - The model, chain, and atom levels of the hierarchy object are probably immediately obvious to someone familiar with the content of model files (such as a PDB file). Note that that is no 'residue type' in the data structure. Instead, there are the two types residue_group and atom_group. They are related to alternative conformations. 

 - If there are no alternative conformations in the model, all residue groups contain exactly one atom group, which contains all the atoms of a residue. A file with alternative conformations will lead to residue groups with multiple atom groups, one for each conformer. (Note: about a quarter of the files in the PDB database contain alternative conformations).


#### Looping over hierarchy

In [ ]:
for m in hierarchy.models(): 
  for chain in m.chains():
    for rg in chain.residue_groups():
      for ag in rg.atom_groups():
        for atom in ag.atoms():
          # your code here
          pass


Many hierarchy objects have methods to directly access objects lower down in the hierarchy as an iterable. Lets see what is in the first four residues (atom groups)

In [ ]:
atom_groups = list(hierarchy.atom_groups())

for ag in atom_groups[:4]:
  rg = ag.parent() # access the residue group
  chain = rg.parent().parent() # access the chain
  print("Chain ID:",chain.id)
  print(" Residue Num:",rg.resseq)
  print(" Residue Name:",ag.resname)
  for atom in ag.atoms():
    print("     Atom:",atom.name)
  

###Example: Truncate to Poly-alanine
We have residue names and atom names now, but to perform the truncation, we need to know which residues are amino acids (because we only want to truncate amino acids), and which atom names we want to keep.
The iotbx.pdb module contains the sub-module amino_acid_codes. This sub-module contains two Python dictionaries, one of which is (shortened):
```
one_letter_given_three_letter = {
"ALA": "A",
"ARG": "R",
...
"TYR": "Y",
"VAL": "V"}
```

We don't need the one-letter codes, but we can re-use the keys of this dictionary to efficently decide if a residue name corresponds to an amino acid.

In [ ]:
import iotbx.pdb.amino_acid_codes
aa_resnames = iotbx.pdb.amino_acid_codes.one_letter_given_three_letter
ala_atom_names = set([" N  ", " CA ", " C  ", " O  ", " CB "])


def truncate_polyala(model):
  model_polyala = model.deep_copy()

  for m in model_polyala.get_hierarchy().models():
    for chain in m.chains():
      for rg in chain.residue_groups():
        for ag in rg.atom_groups():
          if ag.resname in aa_resnames:
            ag.resname = "ALA"
            for atom in ag.atoms():
              if atom.name not in ala_atom_names:
                ag.remove_atom(atom=atom)
      
    return model_polyala

model_polyala = truncate_polyala(model)
show(model_polyala)


#### Improved truncation for rare cases
For most practical purposes, the previous the previous method is sufficient. However, there are some files in the PDB for which this is not true. One example is the structure with the PDB ID 1ysl. Lets see what happens when we run the previous method on a fragment from 1ysl:

**Note:** We are using the same instance of Data Manager here that we used before. The datamanager can store multiple files, accessed by their filename

In [ ]:
filename_1ysl = "resname_mix.pdb"
dm.process_model_file(filename=filename_1ysl)
model_1ysl = dm.get_model(filename=filename_1ysl)

In [ ]:
show(model_1ysl)

Lets try to run our poly-alanine truncation function on this fragment:

In [ ]:
model_1ysl_trunc= truncate_polyala(model_1ysl)
show(model_1ysl_trunc)

It is not all alanine. The reason is because this fragment contains a modified residue CSD (oxidized cysteine). In fact, this residue also has an alternate conformation. Conformation "A" is the modified amino acid CSD, conformation "B" is a regular CYS.

In [ ]:
print(model_1ysl_trunc.model_as_pdb())

Rare cases like this are the reason why the residue_group and atom_group levels are needed in the hierarchy. Here are two different residue names for the same member of a chain. Even though this sitution is rare, it is entirely plausible and valid, and a comprehensive PDB processing library has to be able to handle it.
Our script will only truncate the CYS residue, but it would be better if it also truncated the non-standard CSD residue in the A alternative conformation. Let's find out what it takes to achieve this.

One way would be to add the 3 letter code for all modified amino acids to the dictionary. This however relies on the completeness of the dictionary; what happens if the 3 letter code changes or if other entries are added?
Another possibility is to check if there is at least one amino acid in a residue group, and if so, apply the truncation to all residues in the group, even if they don't have a standard residue name. This means, for each residue group we have to loop over the atom groups twice, first to scan for at least one standard amino-acid residue name, and if there is one, a second time to do the truncation. This will double effort (and computing time), but it is required to accomodate all possible cases.

In [ ]:
import iotbx.pdb.amino_acid_codes
aa_resnames = iotbx.pdb.amino_acid_codes.one_letter_given_three_letter
ala_atom_names = set([" N  ", " CA ", " C  ", " O  ", " CB "])


def truncate_polyala_improved(model):
  model_polyala = model.deep_copy()

  def has_amino_acid(residue_group):
    for ag in residue_group.atom_groups():
      if ag.resname in aa_resnames:
        return True
    return False


  for m in model_polyala.get_hierarchy().models():
    for chain in m.chains():
      for rg in chain.residue_groups():
        if has_amino_acid(rg):
          for ag in rg.atom_groups():
              ag.resname = "ALA"
              for atom in ag.atoms():
                if atom.name not in ala_atom_names:
                  ag.remove_atom(atom=atom)
      
    return model_polyala

model_polyala = truncate_polyala(model)


In [ ]:
model_1ysl_trunc = truncate_polyala_improved(model_1ysl)
show(model_1ysl_trunc)

Success! The fragment was truncated to poly-alanine. Hopefully this section emphasizes the distinction between residue groups and atom groups in the CCTBX hierarchy.

### Selections
In many cases it is desireable to process a subset of a model. Fortunately, the model object supports a variety of string selections. In many cases this can enable you to skip looping over the hierarchy. 

\
For example, to select the first 20 residues:


In [ ]:
model_subset = model.select(model.selection("resseq 1:20"))
show(model_subset)

What is happening here? The inner statement parses the string selection, and turns it into a boolean flex array. Flex arrays are CCTBX numerical arrays implemented in C++. Each value in the array relates to an atom in the model, and the value True/False corresponds to whether or not the atom was selected by the selection string.

In [ ]:
sel = model.selection("resseq 1:20")
print(sel)

We can see how many values are set to True using the count() method on the flex array. We can see the total length of the array using the normal Python len() function.

In [ ]:
print(sel.count(True))
print(len(sel))

This corresponds to 160 atoms selected, out of 880 atoms total. You may recall from the hierarchy.composition() function, there are 880 atoms in this structure.

\
The outer component of our selection statement above, the select() function, takes the boolean flex array and converts it into a new model that only includes the selected atoms. This means that you can construct your own selection array if desired instead of using the string selection.

\


Other options for string selections are documented on this Phenix site:\
 https://phenix-online.org/documentation/reference/atom_selections.html

 \
 For example, say we want to select all aromatic amino acids. This will demonstrate using the logical "or" expression in string selections:

In [ ]:
model_aromatic = model.select(model.selection("resname TYR or resname PHE or resname TRP"))
show(model_aromatic)

There were not any Tryptophans, but we efficiently selected all the Tyrosines and Phenylalanines. We could have done this by looping over the hierarchy, but doing that manually is significantly slower

In [ ]:
%%time
sel = [False]*model.get_number_of_atoms()

for m in model.get_hierarchy().models():
  for chain in m.chains():
    for rg in chain.residue_groups():
      for ag in rg.atom_groups():
        if ag.resname in ["PHE","TYR","TRP"]:
          for atom in rg.atoms():
            sel[atom.i_seq]=True

sel = flex.bool(sel)

In [ ]:
%%time 
sel = model.selection("resname TYR or resname PHE or resname TRP")

Note that the first example, looping over the hierarchy, uses the atom.i_seq property. This is the unique integer sequence value for each atom, and this is the position in the selection array that the atom corresponds to.

#### Flex and Numpy: An example using B-factors and coordinates

Flex arrays are numerical arrays implemented in C++. They have a lot in common with the widely used Numpy arrays. In fact, we can directly access a Numpy array from a Flex array. This is useful for passing data on to other libraries based on Numpy.

In [ ]:
b_iso = model.get_b_iso()
print(b_iso)

In [ ]:
b_iso_np = b_iso.as_numpy_array()
print(type(b_iso_np))

In [ ]:
b_iso_np

In [ ]:
list(b_iso_np)==list(b_iso)

In [ ]:
import numpy as np

np.all(np.array(b_iso) == b_iso_np)

In [ ]:
all(flex.double(b_iso_np)==b_iso)

Both can easily be converted back and forth directly, or through an intermediate Python list. Because of this interoperabiliy, we can easily use any of the Python libraries based on Numpy, for example Matplotlib.

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
_ = plt.hist(b_iso)
plt.title("B-factor histogram")
plt.xlabel("B-factor")
plt.ylabel("Count")

##### Cartesian coordinates
Lets extract the cartesian coordinates for the model. It will be a flex array of N 3D vectors, where N is the number of atoms.

In [ ]:
model_copy = model.deep_copy()
xyz = model_copy.get_sites_cart()
xyz

We will move a fragment of 100 atoms by a translation vector to illustrate setting new coordinates for a model

In [ ]:
translation = (20,20,20)

for i in range(100):
  x,y,z = xyz[i]
  a,b,c = translation
  xyz[i] = x+a, y+b, z+c

In [ ]:
model_copy.set_sites_cart(xyz)

In [ ]:
show(model_copy)

## Model geometry
The final major category of information stored by the model manager is information related to geometry restraints. Geometry restraints provide a framework for evaluating how far away a model is from an idealized model.


In [ ]:
model.process_input_model(make_restraints=True)
model.geometry_statistics().show()

In [ ]:
geometry_restraints = model.get_restraints_manager()
geometry_energies = geometry_restraints.geometry.energies_sites(sites_cart = model.get_sites_cart())
angle_deltas = geometry_energies.angle_proxies.deltas(sites_cart=model.get_sites_cart())
bond_deltas = geometry_energies.bond_proxies.deltas(sites_cart=model.get_sites_cart())

fig,axes = plt.subplots(1,2,figsize=(12,5))
ax1,ax2 = axes
ax1.hist(bond_deltas,bins=50)
ax1.set_xlabel("Bond delta (angstroms)")

ax2.hist(angle_deltas,bins=50)
ax2.set_xlabel("Angle delta (degrees)")


Knowing how far a feature deviates from the ideal provides a way to incorporate molecular geometry into a target function. A large number of restraint types can be managed using the geometry restraints manager:
  - Bonds
  - angles
  - chirality
  - dihedrals
  - planarity
  - secondary structure
  - non-bonded overlaps
  - ramachandran
  - rotamers
  - etc

# Maps

##The Map Model Manager
After models, probably the second most integral component of the CCTBX data structures are density maps. The two data structures we will introduce here are the Map manager and the Map Model Manager. The Map Model Manager in particular is a very high level object with many convienence functions. For example, if we want to generate some test data without reading in files, we can do that.

In [ ]:
from iotbx.map_model_manager import map_model_manager      
mmm=map_model_manager()
mmm.generate_map()

In [ ]:
show(mmm)

This small fragment was taken from a library, and the map was calculated using the model. The generate_map() method has many options that let you choose parameters such as:
  - The number of residues (n_residues)
  - The B-factor (b_iso)
  - The high resolution limit (d_min)
  - The scattering table (scattering_table)
  
It is also possible to supply a model or model file directly. Note that the default choice for the scattering table is “electron”, so it will create a cryo-EM map. If you want an electron density or nuclear density map, the value will have to be modified accordingly. 

\
Let's experiment with the effect of changing the B-factor and resolution for the simulated map.

In [ ]:
d_min = 8
mmm=map_model_manager()
mmm.set_resolution(d_min)
mmm.generate_map(d_min=d_min)
show(mmm)

This will be very familiar to all cryo-electron microscopists. Lets try to get a very nice looking map.

In [ ]:
d_min = 1.0
mmm=map_model_manager()
mmm.set_resolution(d_min)
mmm.generate_map(d_min=d_min,b_iso=10)
show(mmm)

We can play with different values for the isotropic B-factor...

In [ ]:
d_min = 1.0
mmm=map_model_manager()
mmm.set_resolution(d_min)
mmm.generate_map(d_min=d_min,b_iso=80)
show(mmm)

When calculating the map using a higher isotropic B-factor, the density is visually degraded, which is what we expect.

Lets generate a map for the 1aba model we have been working on. We will aim to generate a map at the resolution typical for cryo-EM data. Because we are trying to simulate a cryo-EM map, we need to change the space group to P1



In [ ]:
# 1aba crystal symmetry (Not P1)
print("\n1aba:\n",model.crystal_symmetry())

from cctbx.maptbx.box import shift_and_box_model
model_p1 = shift_and_box_model(model)

# should be P1
print("\n1aba reboxed as P1:\n",model_p1.crystal_symmetry())


You can see that the shift_and_box_model() function made a P1 Unit cell that is big enough to accommodate the model. This is commonly used as the crystal symmetry for cryo-EM maps in CCTBX. Now we will simulate the density.

In [ ]:
mmm=map_model_manager()
mmm.generate_map(model=model_p1,d_min=3)
show(mmm)

##The Map Manager

To access the raw numerical density values, we can access the map_data flex on the Map Manager object

In [ ]:
mm = mmm.map_manager()
map_data = mm.map_data()
map_data

The shape of the map_data array can be accessed using the all() method

In [ ]:
map_data.all()

These are the dimensions of the volume data. They were determined automatically when we generated the map, based on the resolution supplied and the unit cell dimensions. We can retrieve the resulting pixel sizes from the Map Manager as well:

In [ ]:
mm.pixel_sizes()

### Extracting a sub map

Often times it is desireable to only work with the density in one local area, for example, around a ligand. One of the convienent features of the Map Model Manager is that it can do this for you. Lets try to look at the density for the MES ligand.

In [ ]:
ligand_mmm = mmm.extract_all_maps_around_model(selection_string="resname MES")
show(ligand_mmm)

The box_cushion argument controls how large the extracted density is. We can try with a smaller value if we don't need so much density in the neighborhood of our ligand

In [ ]:
ligand_mmm = mmm.extract_all_maps_around_model(selection_string="resname MES",
                                               box_cushion=2.0)
show(ligand_mmm)

Now we can more easily evaluate the local density around the ligand. To adjust the contour level, we can pass different values to the show() function. First lets histogram the density values:

In [ ]:
import matplotlib.pyplot as plt

_ = plt.hist(ligand_mmm.map_data(),bins=30)
plt.xlabel("Map value")

Lets set the value to 0.4

In [ ]:
show(ligand_mmm,map_threshold=0.4)

To finish, we will now save the map to illustrate how to save real space maps using the Data Manager

In [ ]:
dm.write_real_map_file(mm,filename="1aba_calculated_at_3A.mrc")

# Reflections


## Miller array
Miller arrays are containers for experimental (or calculated data). A miller array contains the crystal symmetry, an array of Miller indices (h,k,l), a boolean flag indicating anomalous pairs and a flex array containing data (X-ray amplitudes or intensities, experimental sigmas, etc.).

\
Lets open the reflections for a new structure, 4zyp
 

In [ ]:
filename = "/content/4zyp.mtz"
dm.process_miller_array_file(filename)
reflections_reader = dm.get_miller_array(filename=filename)

The reflections reader is a container which can store multiple miller arrays. Lets build a dictionary of all the miller arrays and the data labels there were present in the file

In [ ]:
miller_array_dict = {ma.info().label_string():ma for ma in reflections_reader.as_miller_arrays()}

for key,value in miller_array_dict.items():
  print(key,":",type(value))

We will choose the FOBS,SIGFOBS array and look at some of the data stored in the miller array

In [ ]:
f_obs = miller_array_dict["FOBS,SIGFOBS"]

In [ ]:
print("Resolution limits:", f_obs.d_max_min())
print("\nSpace group info:",f_obs.space_group_info())
print("\nCompleteness:",f_obs.completeness())
print("\nSize:",f_obs.size())

We can also select the R-free in the MTZ file. Let's have a closer look at that array.

In [ ]:
r_free = miller_array_dict["R-free-flags"]

In [ ]:
print("Resolution limits:", r_free.d_max_min())
print("\nSpace group info:",r_free.space_group_info())
print("\nCompleteness:",r_free.completeness())
print("\nSize:",r_free.size())

The size of the miller arrays FOBS,SIGFOBS and R-free-flags is not the same. In other words, the number of hkl indices with corresponding data or R-free-flag differs in the two arrays. This situation occurs rather frequently, even with deposited data in the PDB. For example, the mismatch can occur when R-free-flags are copied from another data set.

\
Programs dealing with data have to address this mismatch. Let's say we want to calculate R-factors. If a reflection with an associated intensity does not have an R-free flag, it can't be assigned neither to the working set nor the the test set of reflections. It is therefore useful to obtain a common set of reflections. Let's try to obtain such a common set for out test data set.

\
It is instructive to determine how many reflections are mismatched. The miller class has the method match_indices, which will do all the work.

In [ ]:
match_result = f_obs.match_indices(r_free)

In [ ]:
n_common = match_result.pairs().size()
n_fobs_only = f_obs.size()-n_common
n_rfree_only = r_free.size()-n_common
print("Common:",n_common)
print("F-obs only:",n_fobs_only)
print("R-free only:",n_rfree_only)

We just found out that the R-free-flags array has about 500 additional miller indices. Note that match_indices is a low level method and is typicially not used. It is nevertheless instructive for this example. Practically, to obtain a common set of reflections, the common_sets() method can be used:

In [ ]:
f_obs, r_free = f_obs.common_sets(r_free)
print(f_obs.size())
print(r_free.size())

##F-model: Synthesis of model and data
Let's go back to our structure, 1aba. We will include code for imports and to open the model and data mtz file together, to enable starting the examples here.

In [ ]:
from iotbx.data_manager import DataManager

dm = DataManager()
filename_pdb = "1aba_model.pdb"
dm.process_model_file(filename_pdb)
model = dm.get_model(filename=filename_pdb)

filename_mtz = "/content/1aba_reflections.mtz"
dm.process_miller_array_file(filename=filename_mtz)
reflections_reader = dm.get_miller_array(filename=filename_mtz)


And select F-obs and R-free miller arrays

In [ ]:
miller_array_dict = {ma.info().label_string():ma for ma in reflections_reader.as_miller_arrays()}
print(miller_array_dict.keys())

In [ ]:
# select fobs and rfree
f_obs = miller_array_dict['FOBS,SIGFOBS']
r_free = miller_array_dict['FreeR_flag']
f_obs, r_free = f_obs.common_sets(r_free)

We will instantiate an F Model Manager object. This combines the data and the structure together. Note that the model we will used is an xray structure object. The xray structure object is a reduced form of the model, containing only the information about the scatterers which is needed for calculations.

In [ ]:
import mmtbx

xrs = model.get_xray_structure()
r_free_flags = r_free.array(data = r_free.data()==0)

f_model = mmtbx.f_model.manager(
  f_obs          = f_obs,
  r_free_flags   = r_free_flags,
  xray_structure = xrs)

The update_all_scales() method is necessary to calculate scale factors and add a bulk solvent model. This procedure puts the calculated and observed structure factor amplitudes to a common scale and adjusts calculated low resolution structure factors.

In [ ]:
_ = f_model.update_all_scales()

Now we will print out the R-work and R-free values

In [ ]:
print("R-work:",f_model.r_work())
print("R-free:",f_model.r_free())

Now perform a fast-Fourier-transform (FFT) to convert the structure factors into a real map

In [ ]:
f_calc = f_model.f_calc()
fft_map = f_calc.fft_map()
fft_map.apply_sigma_scaling()
map_data = fft_map.real_map_unpadded()

Now we can instantiate a map manager manually from the three basic components:
  - Map data
  - Unit cell grid
  - Unit cell crystal symmetry

In [ ]:
from iotbx.map_manager import map_manager

mm=map_manager(map_data=map_data,
                unit_cell_grid=map_data.all(), # the shape of the map array
                unit_cell_crystal_symmetry=xrs.crystal_symmetry(),
                wrapping=True)

In [ ]:
from cctbx_py3dmol_wrapper import CCTBX3dmolWrapper
show = CCTBX3dmolWrapper()
show(mm)

Of course, this is an F-calc map, so it is similar to what we got before using the generate_map() method. Let's try to calculate the 2mFo-DFc map.

In [ ]:
from mmtbx import map_tools

map_coeff = map_tools.electron_density_map(fmodel = f_model).map_coefficients(map_type="2mFo-DFc")
fft_map = map_coeff.fft_map()
fft_map.apply_sigma_scaling()
map_data = fft_map.real_map_unpadded()

We can instantiate a Map Model Manager with the maps and the model

In [ ]:
from iotbx.map_manager import map_manager
from iotbx.map_model_manager import map_model_manager

mm_2fofc=map_manager(map_data=map_data,
                unit_cell_grid=map_data.all(),
                unit_cell_crystal_symmetry=xrs.crystal_symmetry(),
                wrapping=True)

mmm_2fofc = map_model_manager(model=model,map_manager=mm_2fofc)
ligand_mmm_2fofc = mmm_2fofc.extract_all_maps_around_model(selection_string="resname MES")
show(ligand_mmm_2fofc,map_threshold=0.4)